# Xarray-spatial
### User Guide: Cost Distance (Weighted Proximity)
-----

The `cost_distance` function computes the **minimum accumulated traversal cost** through a friction surface to reach the nearest target pixel. This is the raster equivalent of GRASS `r.cost` / ArcGIS *Cost Distance*.

Unlike `proximity`, which measures geometric (straight-line) distance ignoring terrain, `cost_distance` accounts for a **friction surface** — a raster where each cell's value represents how costly it is to traverse that cell.

**Contents:**
- [Setup](#Setup)
- [1. Uniform friction: cost_distance vs proximity](#1.-Uniform-friction:-cost_distance-vs-proximity)
- [2. Variable friction surface](#2.-Variable-friction-surface)
- [3. Barriers (impassable cells)](#3.-Barriers-(impassable-cells))
- [4. max_cost truncation](#4.-max_cost-truncation)
- [5. Multiple sources](#5.-Multiple-sources)
- [6. 4-connectivity vs 8-connectivity](#6.-4-connectivity-vs-8-connectivity)
- [7. Dask support for large rasters](#7.-Dask-support-for-large-rasters)

-----

## Setup

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from xrspatial import proximity, cost_distance

In [ ]:
def make_raster(data, res=1.0):
    """Helper: create a DataArray with y/x coordinates."""
    h, w = data.shape
    raster = xr.DataArray(
        data.astype(np.float64),
        dims=['y', 'x'],
        attrs={'res': (res, res)},
    )
    raster['y'] = np.arange(h) * res
    raster['x'] = np.arange(w) * res
    return raster


def plot_comparison(arrays, titles, cmaps=None, figsize=None):
    """Plot multiple arrays side by side."""
    n = len(arrays)
    if figsize is None:
        figsize = (5 * n, 4)
    if cmaps is None:
        cmaps = ['viridis'] * n
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, arr, title, cmap in zip(axes, arrays, titles, cmaps):
        data = arr.values if hasattr(arr, 'values') else arr
        im = ax.imshow(data, cmap=cmap, origin='upper')
        ax.set_title(title)
        plt.colorbar(im, ax=ax, shrink=0.8)
    plt.tight_layout()
    plt.show()

## 1. Uniform friction: cost_distance vs proximity

When friction is uniform (all 1s), `cost_distance` approximates Euclidean `proximity` — the accumulated cost equals the geometric distance along the grid path.

The two functions use different algorithms: `proximity` computes true Euclidean (straight-line) distance via a 4-pass DP sweep, while `cost_distance` uses Dijkstra on an 8-connected grid. On a grid, the shortest 8-connected path is slightly longer than the true Euclidean distance for non-axis-aligned directions (a well-known grid discretization artifact). The maximum error is bounded by `(sqrt(2) - 1) * cellsize` per step, and typically under 1 unit for small grids.

In [ ]:
# Create a 21x21 grid with a single source at the centre
source = np.zeros((21, 21))
source[10, 10] = 1.0

raster = make_raster(source)
friction_uniform = make_raster(np.ones((21, 21)))

# Compute both
prox_result = proximity(raster)
cost_result = cost_distance(raster, friction_uniform)

plot_comparison(
    [prox_result, cost_result, prox_result - cost_result],
    ['proximity (Euclidean)', 'cost_distance (friction=1)', 'Difference'],
    cmaps=['magma', 'magma', 'RdBu'],
)

In [ ]:
# Grid discretization artifact: Dijkstra on an 8-connected grid overestimates
# true Euclidean distance for non-axis-aligned directions.
# For a 21x21 grid this is well under 1 unit.
diff = np.abs(prox_result.values - cost_result.values)
print(f"Max absolute difference: {np.nanmax(diff):.6f}")
print(f"Mean absolute difference: {np.nanmean(diff):.6f}")

# Cardinal and diagonal distances match exactly:
print(f"\nCardinal neighbour — proximity: {prox_result.values[10, 11]:.4f}, cost_distance: {cost_result.values[10, 11]:.4f}")
print(f"Diagonal neighbour — proximity: {prox_result.values[9, 9]:.4f}, cost_distance: {cost_result.values[9, 9]:.4f}")

## 2. Variable friction surface

Now let's add a **high-friction zone** (like a river, swamp, or dense forest) across the middle of the grid. `proximity` ignores this entirely, but `cost_distance` routes around it because traversal is expensive.

In [ ]:
# Build a friction surface with a high-cost band across rows 8-12
friction_data = np.ones((21, 21))
friction_data[8:13, :] = 10.0  # 10x more costly to cross this zone

# Source at top-left corner
source2 = np.zeros((21, 21))
source2[2, 2] = 1.0

raster2 = make_raster(source2)
friction_var = make_raster(friction_data)

prox2 = proximity(raster2)
cost2 = cost_distance(raster2, friction_var)

plot_comparison(
    [friction_var, prox2, cost2],
    ['Friction surface', 'proximity (ignores friction)', 'cost_distance'],
    cmaps=['YlOrRd', 'magma', 'magma'],
)

Notice how:
- `proximity` shows smooth concentric circles centred on the source — it doesn't "see" the friction band.
- `cost_distance` shows the cost jump across the high-friction band. Pixels on the far side have much higher accumulated cost because every path must traverse that expensive zone.

## 3. Barriers (impassable cells)

Setting friction to **NaN** or **0** makes cells completely impassable. `cost_distance` will route around barriers; if pixels are fully cut off, they get NaN.

Compare with `proximity`, which always uses straight-line distance through everything.

In [ ]:
# Create a wall with a narrow gap
friction_barrier = np.ones((21, 21))
friction_barrier[5:16, 10] = np.nan   # vertical wall
friction_barrier[10, 10] = 1.0         # gap in the wall at row 10

# Source on the left side
source3 = np.zeros((21, 21))
source3[10, 3] = 1.0

raster3 = make_raster(source3)
friction_wall = make_raster(friction_barrier)

prox3 = proximity(raster3)
cost3 = cost_distance(raster3, friction_wall)

# Show friction surface with barrier visible
barrier_vis = friction_barrier.copy()
barrier_vis[np.isnan(barrier_vis)] = 0  # show barrier as 0 for visualisation

plot_comparison(
    [make_raster(barrier_vis), prox3, cost3],
    ['Friction (dark = barrier)', 'proximity (ignores wall)', 'cost_distance (routes through gap)'],
    cmaps=['gray', 'magma', 'magma'],
    figsize=(15, 4),
)

The `cost_distance` result shows that all paths to the right side must funnel through the single gap in the wall, creating asymmetric cost contours.

## 4. max_cost truncation

The `max_cost` parameter limits the search radius. Pixels whose cheapest path exceeds the budget are set to NaN. This is critical for Dask scalability — it bounds the overlap region needed for each chunk.

In [ ]:
source4 = np.zeros((21, 21))
source4[10, 10] = 1.0

raster4 = make_raster(source4)
friction4 = make_raster(np.ones((21, 21)))

cost_full = cost_distance(raster4, friction4)
cost_limited = cost_distance(raster4, friction4, max_cost=6.0)

plot_comparison(
    [cost_full, cost_limited],
    ['cost_distance (no limit)', 'cost_distance (max_cost=6)'],
    cmaps=['magma', 'magma'],
)

Pixels beyond cost 6 are shown as white (NaN). This is useful for questions like "which areas are reachable within a given travel budget?"

## 5. Multiple sources

When multiple target pixels exist, `cost_distance` finds the cheapest path to the **nearest** source for each pixel — just like `proximity` finds the nearest by geometric distance.

With non-uniform friction, the "nearest" source by cost may differ from the nearest by straight-line distance.

In [ ]:
# Two sources, with a high-friction zone near one of them
source5 = np.zeros((21, 21))
source5[5, 5] = 1.0    # source A (top-left region)
source5[15, 15] = 2.0  # source B (bottom-right region)

friction5_data = np.ones((21, 21))
friction5_data[3:8, 3:8] = 5.0  # high friction around source A
friction5_data[5, 5] = 1.0      # but source A itself is cheap to stand on

raster5 = make_raster(source5)
friction5 = make_raster(friction5_data)

prox5 = proximity(raster5)
cost5 = cost_distance(raster5, friction5)

plot_comparison(
    [friction5, prox5, cost5],
    ['Friction (high near source A)', 'proximity', 'cost_distance'],
    cmaps=['YlOrRd', 'magma', 'magma'],
)

Notice how the cost-distance boundary between the two sources shifts: the high friction around source A makes source B "closer" (by cost) to more of the grid than geometric distance alone would suggest.

## 6. 4-connectivity vs 8-connectivity

By default, `cost_distance` uses 8-connectivity (cardinal + diagonal neighbours). With `connectivity=4`, only cardinal neighbours (up/down/left/right) are considered. This means diagonal paths must take two steps instead of one, increasing costs along diagonals.

In [ ]:
source6 = np.zeros((15, 15))
source6[7, 7] = 1.0

raster6 = make_raster(source6)
friction6 = make_raster(np.ones((15, 15)))

cost_8conn = cost_distance(raster6, friction6, connectivity=8)
cost_4conn = cost_distance(raster6, friction6, connectivity=4)

plot_comparison(
    [cost_8conn, cost_4conn, cost_4conn - cost_8conn],
    ['8-connectivity', '4-connectivity', 'Difference (4-conn minus 8-conn)'],
    cmaps=['magma', 'magma', 'Reds'],
)

In [ ]:
# The corner pixel shows the biggest difference
print(f"Corner (0,0) with 8-connectivity: {cost_8conn.values[0, 0]:.4f}")
print(f"Corner (0,0) with 4-connectivity: {cost_4conn.values[0, 0]:.4f}")
print(f"Ratio: {cost_4conn.values[0, 0] / cost_8conn.values[0, 0]:.4f}")

## 7. Dask support for large rasters

For very large rasters that don't fit in memory, `cost_distance` works with Dask-backed DataArrays. When `max_cost` is finite, it automatically computes the required overlap radius and uses `dask.array.map_overlap` for parallel, chunk-by-chunk processing.

The key formula: `overlap_radius = max_cost / (min_friction * cellsize)`

In [ ]:
import dask.array as da

# Create a larger raster backed by dask
np.random.seed(42)
h, w = 100, 100

source_data = np.zeros((h, w))
source_data[20, 20] = 1.0
source_data[80, 80] = 2.0

friction_data = np.random.uniform(1.0, 3.0, (h, w))

raster_dask = make_raster(source_data)
raster_dask.data = da.from_array(source_data, chunks=(50, 50))

friction_dask = make_raster(friction_data)
friction_dask.data = da.from_array(friction_data, chunks=(50, 50))

print(f"Raster type: {type(raster_dask.data)}")
print(f"Chunks: {raster_dask.data.chunks}")

In [ ]:
# Compute with max_cost to enable efficient chunked processing
result_dask = cost_distance(raster_dask, friction_dask, max_cost=80.0)

print(f"Result type: {type(result_dask.data)}")
print(f"Result chunks: {result_dask.data.chunks}")

# Compute to numpy for plotting
result_computed = result_dask.compute()

plot_comparison(
    [make_raster(friction_data), result_computed],
    ['Random friction surface', 'cost_distance (dask, max_cost=80)'],
    cmaps=['YlOrRd', 'magma'],
)

In [ ]:
# Verify dask matches numpy
raster_np = make_raster(source_data)
friction_np = make_raster(friction_data)
result_np = cost_distance(raster_np, friction_np, max_cost=80.0)

max_diff = np.nanmax(np.abs(result_computed.values - result_np.values))
print(f"Max difference between dask and numpy: {max_diff:.10f}")

## Summary

| Function | What it measures | Friction-aware? | Algorithm |
|---|---|---|---|
| `proximity` | Geometric distance to nearest target | No | 4-pass DP (GDAL) |
| `cost_distance` | Accumulated traversal cost to nearest target | Yes | Multi-source Dijkstra |

**When to use which:**
- Use `proximity` when you need simple geometric distance (e.g., "how far is each pixel from the nearest road?").
- Use `cost_distance` when traversal cost varies across the landscape (e.g., "what is the cheapest path cost from each pixel to the nearest hospital, accounting for terrain difficulty?").

**Key parameters for `cost_distance`:**
- `friction`: A raster where each cell's value represents the per-unit-distance cost of traversal. NaN or ≤0 = impassable.
- `max_cost`: Limits the search radius. Critical for Dask scalability.
- `connectivity`: 4 (cardinal only) or 8 (cardinal + diagonal).
- `target_values`: Specify which pixel values in the source raster are targets.

### References
- GRASS GIS `r.cost`: https://grass.osgeo.org/grass-stable/manuals/r.cost.html
- ArcGIS Cost Distance: https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/cost-distance.htm